In [2]:
import numpy as np
import soundfile
import torch
import src.wavmark as wavmark
import matplotlib.pyplot as plt
from src.wavmark.utils.wm_add_util import fix_pattern

from IPython.display import Audio

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = wavmark.load_model().to(device)

In [5]:
pattern_bits = 16
payload = np.random.choice([0, 1], size=32 - pattern_bits)
# payload = fix_pattern[pattern_bits:]
print(f"Payload: {payload}")
print(f"Pattern: {fix_pattern[:pattern_bits]}")

Payload: [0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1]
Pattern: [1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0]


In [18]:
# load audio
audio_path = "/home/zrh/Downloads/VALL-E_1.wav"
signal, sample_rate = soundfile.read(audio_path)
Audio(signal, rate=sample_rate)

In [19]:
# encode
watermarked_signal, info_encode = wavmark.encode_watermark(model, signal, payload, pattern_bit_length=pattern_bits, show_progress=True)
print(info_encode)

Processing: 100%|██████████| 3/3 [00:00<00:00, 10.79it/s]

{'time_cost': 0.2793867588043213, 'encoded_sections': 3, 'skip_sections': 0, 'snr': 37.67267105558329}


In [20]:
# show watermarked audio
Audio(watermarked_signal, rate=sample_rate)

In [21]:
# decode
watermarked_signal_noise = watermarked_signal

payload_decoded, _ = wavmark.decode_watermark(model, watermarked_signal_noise, len_start_bit=pattern_bits, show_progress=True)
BER = (payload != payload_decoded).mean() * 100

100%|██████████| 7/7 [00:03<00:00,  1.78it/s]


In [22]:
print(payload_decoded, BER)

[0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 1] 0.0


In [4]:
# decode an audio without watermark
audio_path = "/home/zrh/Downloads/VALL-E_1.wav"
signal, sample_rate = soundfile.read(audio_path)
Audio(signal, rate=sample_rate)

In [7]:
payload_decoded, _ = wavmark.decode_watermark(model, signal, len_start_bit=16, show_progress=True)
print(payload_decoded)

100%|██████████| 7/7 [00:04<00:00,  1.53it/s]

None
